# Hsp70 protein Notebook
The Hsp70 protein is chaperone protein; they can assist folding and assembly of newly synthesized proteins, trigger refolding cycles of misfolded proteins, transport unfolded proteins through organelle membranes, and when necessary, deliver non-functional proteins to the proteasome, endosome or lysosome for recycling.
Hsp70 genes differ by organism, location of expression (Nucleus/Cytoplasm, Mitochondria, ER, Chloroplasta), mode of expression (stress-induced or constitutive), substrate specificity (Target regular proteins or Iron-Sulfur cluster proteins,...). Here, RBM are used to highlight automatically the main differences between the subfamilies.



# Loading data and packages

In [ ]:
%matplotlib inline
import sys,os,pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append('RBM/')
sys.path.append('utilities/')

try:
    import rbm
except:
    print 'Compiling cy_utilities first' # the RBM package contains cython files that must be compiled first.
    curr_dir = os.getcwd()
    os.chdir('RBM/')
    !python setup.py build_ext --inplace
    print 'Compilation done'
    os.chdir(curr_dir)
    import rbm


import Proteins_utils, Proteins_RBM_utils, utilities,sequence_logo,plots_utils


filename = 'Hsp70_protein_MSA.fasta'
path = u'data/Hsp70/'

all_data, all_labels = Proteins_utils.load_FASTA(path+filename, with_labels = True)

env = pickle.load(open(path+'Hsp70_info.data'))
for key,item in env.items():
    globals()[key] = item

/users/tubiana/anaconda/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/users/tubiana/anaconda/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/users/tubiana/anaconda/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/users/tubiana/anaconda/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/users/tubiana/anaconda/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may i

# Training

In [ ]:
make_training = True

n_v = 675 # Number of visible units; = # sites in alignment.
n_h = 200 # Number of hidden units.
visible = 'Potts' # Nature of visible units potential. Here, Potts states...
n_cv = 21 # With n_cv = 21 colors (all possible amino acids + gap)
hidden = 'dReLU' # Nature of hidden units potential. Here, dReLU potential.
seed = 0 # Random seed (optional)

if make_training: # Make full training.
    RBM = rbm.RBM(visible = visible,hidden = hidden,n_v = n_v,n_h = n_h, n_cv = n_cv, random_state = seed)
    batch_size = 100 # Size of mini-batches (and number of Markov chains used). Default: 100. Value for RBM shown in paper: 300
    n_iter = 305 # Number of epochs. Value for RBM shown in paper: 305
    learning_rate = 0.1 # Initial learning rate (default: 0.1). Value for RBM shown in paper: 0.1
    decay_after = 0.33 # Decay learning rate after 33% of iterations (default: 0.5). Value for RBM shown in paper: 0.33
    l1b = 0.8 # L1b regularization. Default : 0. Value for RBM shown in paper: 0.8
    N_MC = 1 # Number of Monte Carlo steps between each update. Value for RBM shown in paper: 1

    RBM.fit(all_data[in_train][order_train], weights= all_weights[in_train][order_train], batch_size = batch_size,
        n_iter = n_iter, l1b = l1b, N_MC = N_MC, 
       decay_after = decay_after, verbose = 0 )
else:
    RBM = Proteins_RBM_utils.loadRBM('models/RBM_Hsp70_Protein.data') ## Alternative: Load previous model (unzip first).


Starting epoch 1
Starting epoch 2
Starting epoch 198
Starting epoch 199
Starting epoch 200
Starting epoch 201
Starting epoch 202
Starting epoch 203
Starting epoch 204
Starting epoch 205
Starting epoch 206
Starting epoch 207
Starting epoch 208
Starting epoch 209


## Visualizing hidden units: Sequence logo of weights
Show sequence logos of some selected weights. Here, features reflect the structural constraint, as well as phylogenic diversity.


- Feature 1: Short loop on the NBD, discriminating between Prokaryotic DnaK and Eukaryotic Mitochondrial/Chloroplastal Hsp70 from Prokaryotic HSc.A and other Eukaryotic Hsp70. Linked to ATP exchange rate and  NEF co-chaperone specificity.
- Feature 2: A short motif on the beta strand of the SBD, putatively linked to substrate specificity.
- Feature 3: An interdomain feature, at the interface between the LID and SBD domains.
- Feature 4: An interdomain feature, at the interface between the NBD and SBD domains. Discriminates non-allosteric Hsp70 (aka Hsp110) from the others.
- Feature 5: A feature localized on the unstructured tail of the protein. Putatively linked to DnaJ co-chaperone specificity.
- Feature 6: Very Short loop on the NBD, discriminating between Prokaryotic HScA and the rest. Linked to ATP exchange rate and  NEF co-chaperone specificity.
- Feature 7: A variant of the NBD loop for Prokaryotic DNaKs. Putatively linked to NEF specificity.
- Feature 8: An interdomain feature discriminating Eukaryotic Hsp70 expressed in Endoplasmic Reticulum from the rest.
- Feature 9: Another interdomain feature discriminating non-allosteric Hsp70 from the others.
- Feature 10: A 'Dimeric' feature, whose important sites are separated in the ATP/ADP conformation, but close in the dimer.


In [ ]:
if make_training:
    interesting = [100, # short loop, NBD
                   12, # beta-strans, SBD
                   89, # LID-SBD interdomain
                   160, # NBD-SBD interdomain, Allosteric-specific unit
                   186, # Unstructured tail
                   91, # Very short loop, NBD
                   137, # long loop variant, NBD
                   2, # ER-specific unit
                   170, # Allosteric-specific unit #2
                   88] # Dimer mode
else:
    interesting = [3, # short loop, NBD
                   74, # beta-strans, SBD
                   120, # LID-SBD interdomain
                   52, # NBD-SBD interdomain, Allosteric-specific unit
                   71, # Unstructured tail
                   183, # Very short loop, NBD
                   137, # long loop variant, NBD
                   139, # ER-specific unit
                   152, # Allosteric-specific unit #2
                   88] # Dimer mode

fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[0]],ylabel = 'Weights #1',theta_important=0.4);
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[1]],ylabel = 'Weights #2',selected = range(450,475));
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[2]],ylabel = 'Weights #3',theta_important=0.4,nrows=3);
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[3]],ylabel = 'Weights #4',theta_important=0.4,nrows=3);
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[4]],ylabel = 'Weights #5',theta_important=0.4,nrows=1);
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[5]],ylabel = 'Weights #6',theta_important=0.4,nrows=1);
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[6]],ylabel = 'Weights #7',theta_important=0.4,nrows=1);
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[7]],ylabel = 'Weights #8',theta_important=0.4,nrows=3);
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[8]],ylabel = 'Weights #9',theta_important=0.4,nrows=3);
fig, _ = sequence_logo.Sequence_logo_breaks(RBM.weights[interesting[9]],ylabel = 'Weights #10',theta_important=0.4,nrows=3);



## Visualizing hidden units: Distribution of inputs and non-linearity.
Protein sequences cluster according to the projections onto the weights.

In [ ]:
I = RBM.input_hiddens(all_data) # compute hidden unit input.
# Show inputs histogram and conditional means
plots_utils.plot_input_mean(RBM,I, interesting,ncols=2); 

## Visualizing hidden units: Phylogenic activity.
Are the features active across all, or only a portion of the phylogenic tree ? To assess this, we pick for each hidden unit the 20 sequences with highest $I_\mu$ (or lowest, depending on the sign of the non-linearity), and compute the distances between each pair. We compare to the background distribution of pairwise distances

In [ ]:
# Compute histogram of distances between top-activating sequences.
plots_utils.plot_top_activating_distance(RBM, I,all_data,interesting,ncols=2);

# Identification of Functional Subgroups 


In [ ]:
## Scatter plot: [short/long loop] vs [SBD beta strand]
I_background = I

classified = is_dnak | is_hsca | is_nucleus_cyto | is_mitochondrial | is_chloroplastal | is_ER | is_non_allosteric
print classified.sum()



labels_classes = np.asarray(is_dnak * 0 + is_hsca * 1 + is_nucleus_cyto * 2 +is_ER * 5 + is_non_allosteric*6+ \
+ is_mitochondrial * 3 + is_chloroplastal*4,dtype='int')

labels_classes = labels_classes[classified]
I_test = I_background[classified]


labels_names = ['DnaK (P)','HscA (P)','Nucleus/Cyto (E)', 'Mitochondria (E)',
                'Chloroplasta (E)', 'ER (E)','Non-allosteric (E)']




axis_labels1 = [r'$I_{%s}$'%(i+1) for i in range(2)]
axis_labels2 = ['Short Loop', 'SBD beta strand']
axis_labels = [[x + ' (' + y +')' for x,y in zip(axis_labels1,axis_labels2)]]



fig = plots_utils.plot_input_classes_scatter([[interesting[0],interesting[1]]],I_background, I_test, labels_classes,  label_names=labels_names, axis_labels=axis_labels,
              figsize=[8,6], fontsize=20,ncols_legend=2,xlim=None,ylim=[[-12,15]],label_background='UniprotKB');



In [ ]:
## All input histograms
ncols = 3

xlabels1 = [r'$I_{%s}$'%(i+1) for i in range(10)]
xlabels2 = ['Short Loop', 'SBD beta strand', 'LID/SBD','Non-allosteric','Unstructured tail',
           'Very short loop','Loop variant','ER-specific','Non-allosteric 2', 'Dimer']

xlabels = [x + ' (' + y +')' for x,y in zip(xlabels1,xlabels2)]


fig = plots_utils.plot_input_classes(interesting,I_background,I_test,labels_classes,
                                     label_names=labels_names,nbins=25,label_background='UniprotKB',
                                     ncols=3,xlabels=xlabels);



In [ ]:
## Some input scatter plots

liste_interestings = [(0,1),
                     (0,5),
                     (0,6),
                     (0,7),
                     (1,2),
                     (1,9),
                     (3,8),
                     (3,2),
                     (3,9)]

interestings = [ (interesting[linteresting[0]],interesting[linteresting[1]]) for linteresting in liste_interestings ]


xlabels1 = [r'$I_{%s}$'%(i+1) for i in range(10)]
xlabels2 = ['Short Loop', 'SBD beta strand', 'LID/SBD','Non-allosteric','Unstructured tail',
           'Very short loop','Loop variant','ER-specific','Non-allosteric 2', 'Dimer']

xlabels = [x + ' (' + y +')' for x,y in zip(xlabels1,xlabels2)]

axis_labels = [(xlabels[linteresting[0]],xlabels[linteresting[1]]) for linteresting in liste_interestings]


fig = plots_utils.plot_input_classes_scatter(interestings,I_background, I_test, labels_classes,  label_names=labels_names,
              figsize=5, fontsize=15,ncols=3,label_background='UniprotKB',
              axis_labels = axis_labels);


In [ ]:
# Allosteric vs non-allosteric (Input I4)
labels_classes = np.asarray(is_non_allosteric*1,dtype='int')
labels_classes = labels_classes[classified]
labels_names = ['Allosteric','Non-allosteric']
ncols = 1
xlabels1 = [r'$I_{%s}$'%(i+1) for i in range(3,4)]
xlabels2 = ['Non-allosteric']

xlabels = [x + ' (' + y +')' for x,y in zip(xlabels1,xlabels2)]

I_test = I_background[classified]

fig = plots_utils.plot_input_classes([interesting[3]],I_background,I_test,labels_classes,label_names=labels_names,nbins=25,label_background='UniprotKB',ncols=1,xlabels=xlabels);




# A null model for the tail: Check enrichment of the tail in hydrophilic or hydrophobic amino-acids.
Do we find sequences with an hydrophilic-rich (resp. tiny-rich) unstructured tail, or they are expected from an independent model with variable tail length ?
We compare observed statistics with a Monte Carlo simulation with null model...

In [ ]:

surface_sites = np.nonzero(np.abs(RBM.weights[interesting[4]]).sum(-1) > 0.5)[0]
num_sites = len(surface_sites)

seq_num_sites = (all_data[:,surface_sites] <> 20).sum(1)
seq_num_tiny =  ((all_data[:,surface_sites] == Proteins_utils.aadict['A']) | (all_data[:,surface_sites] == Proteins_utils.aadict['G']) ).sum(-1)

seq_num_hydrophilic =  ((all_data[:,surface_sites] == Proteins_utils.aadict['E']) \
| (all_data[:,surface_sites] == Proteins_utils.aadict['K']) | (all_data[:,surface_sites] == Proteins_utils.aadict['R'])\
| (all_data[:,surface_sites] == Proteins_utils.aadict['S']) | (all_data[:,surface_sites] == Proteins_utils.aadict['N'])\
| (all_data[:,surface_sites] == Proteins_utils.aadict['T']) ).sum(-1)




seq_frac_hydrophilic = seq_num_hydrophilic/(1e-10+seq_num_sites)
seq_frac_tiny = seq_num_tiny/(1e-10+seq_num_sites)

lengths = np.arange(1,num_sites+1)
probas = np.array([(seq_num_sites == length).sum() for length in lengths])
probas= 1.0 * probas/probas.sum()



mu_hydrophilic = (seq_frac_hydrophilic ).mean()
mu_tiny = (seq_frac_tiny ).mean()

MC = 50000

all_mus = []
all_weights = []
all_lengths = []
for length,proba in zip(lengths,probas): # Null model with
    print length
    for _ in range(MC):
        rng = np.random.rand(length)
        content = np.zeros([length,1],dtype='int')
        content[rng< mu_hydrophilic] = 0
        content[(mu_hydrophilic <= rng) & (rng< mu_tiny +mu_hydrophilic)] = 1
        content[rng>mu_hydrophilic+mu_tiny] = 2
        mu = utilities.average(content,c=3)[0,:2]
        all_mus.append(mu)
        all_weights.append(proba)
        all_lengths.append(length)

all_mus = np.array(all_mus)
all_weights = np.array(all_weights)


fontsize = 12
fontsize2 = 10
vmax = 20
vmax2 = 0.25

fig, ax = plt.subplots(3,3)
fig.set_figheight(12)
fig.set_figwidth(12)


ax[0,0].hist( [seq_frac_hydrophilic ,all_mus[:,0]],weights=[np.ones(all_data.shape[0]),all_weights],bins=num_sites,normed=True ); 
ax[0,0].legend(['HSP70','Null model'],fontsize=fontsize2);
ax[0,0].set_title('Fraction of hydrophilic amino-acids',fontsize=fontsize);

ax[0,1].hist( [seq_frac_tiny, all_mus[:,1]],weights=[np.ones(all_data.shape[0]),all_weights],bins=num_sites,normed=True );
ax[0,1].legend(['HSP70','Null model'],fontsize=fontsize2);
ax[0,1].set_title('Fraction of tiny amino-acids',fontsize=fontsize);


ax[0,2].hist( [seq_num_sites, all_lengths],weights=[np.ones(all_data.shape[0]),all_weights],bins=num_sites,normed=True );
ax[0,2].legend(['HSP70','Null model'],fontsize=fontsize2);
ax[0,2].set_title('Tail length',fontsize=fontsize);



counts,xbins,ybins,image = ax[1,0].hist2d(seq_frac_hydrophilic,seq_frac_tiny,bins=40,normed=True, range=[(0,1),(0,1)]);
ax[1,0].contour(counts.T,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],linewidths=3,vmin=0,vmax=vmax);
ax[1,0].set_xlim([0,1])
ax[1,0].set_ylim([0,1])
ax[1,0].set_xlabel('Fraction of hydrophilic amino-acids',fontsize=fontsize)
ax[1,0].set_ylabel('Fraction of tiny amino-acids',fontsize=fontsize)
ax[1,0].set_title('Tail content: HSP70')


counts,xbins,ybins,image = ax[1,1].hist2d(seq_frac_hydrophilic,seq_num_sites,bins=[20,25],normed=True, range=[(0,1),(1,num_sites)]);
ax[1,1].contour(counts.T,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],linewidths=3,vmin=0,vmax=vmax2);
ax[1,1].set_xlim([0,1])
ax[1,1].set_ylim([1,num_sites])
ax[1,1].set_xlabel('Fraction of hydrophilic amino-acids',fontsize=fontsize)
ax[1,1].set_ylabel('Tail length',fontsize=fontsize)
ax[1,1].set_title('Tail content: HSP70')


counts,xbins,ybins,image = ax[1,2].hist2d(seq_frac_tiny,seq_num_sites,bins=[20,25],normed=True, range=[(0,1),(1,num_sites)]);
ax[1,2].contour(counts.T,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],linewidths=3,vmin=0,vmax=vmax2);
ax[1,2].set_xlim([0,1])
ax[1,2].set_ylim([1,num_sites])
ax[1,2].set_xlabel('Fraction of tiny amino-acids',fontsize=fontsize)
ax[1,2].set_ylabel('Tail length',fontsize=fontsize)
ax[1,2].set_title('Tail content: HSP70')


counts,xbins,ybins,image = ax[2,0].hist2d(all_mus[:,0],all_mus[:,1],weights=all_weights,bins=40,normed=True, range=[(0,1),(0,1)]);
ax[2,0].contour(counts.T,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],linewidths=3,vmin=0,vmax=vmax);
ax[2,0].set_xlim([0,1])
ax[2,0].set_ylim([0,1])
ax[2,0].set_xlabel('Fraction of hydrophilic amino-acids',fontsize=fontsize)
ax[2,0].set_ylabel('Fraction of tiny amino-acids',fontsize=fontsize)
ax[2,0].set_title('Tail content: Null model')



counts,xbins,ybins,image = ax[2,1].hist2d(all_mus[:,0],all_lengths,weights=all_weights,bins=[20,25],normed=True, range=[(0,1),(1,num_sites)]);
ax[2,1].contour(counts.T,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],linewidths=3,vmin=0,vmax=vmax2);
ax[2,1].set_ylim([1,num_sites])
ax[2,1].set_xlim([0,1])
ax[2,1].set_ylabel('Tail length',fontsize=fontsize)
ax[2,1].set_xlabel('Fraction of hydrophilic amino-acids',fontsize=fontsize)
ax[2,1].set_title('Tail content: Null model')

counts,xbins,ybins,image = ax[2,2].hist2d(all_mus[:,1],all_lengths,weights=all_weights,bins=[20,25],normed=True, range=[(0,1),(1,num_sites)]);
ax[2,2].contour(counts.T,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],linewidths=3,vmin=0,vmax=vmax2);
ax[2,2].set_ylim([1,num_sites])
ax[2,2].set_xlim([0,1])
ax[2,2].set_ylabel('Tail length',fontsize=fontsize)
ax[2,2].set_xlabel('Fraction of tiny amino-acids',fontsize=fontsize)
ax[2,2].set_title('Tail content: Null model')



plt.tight_layout()